# Dimensionality reduction


Importing libraries

In [1]:

#importing libraries
from config import *
import spotipy
import json
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd
import numpy as np
import random
import pprint
import pickle
from sklearn import datasets 
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from matplotlib import pyplot
from sklearn.metrics import silhouette_score
from IPython.display import IFrame



In [2]:
#Initialize SpotiPy with user credentias #
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id=client_id,
                                                           client_secret=client_secret))

## Loading data files

In [3]:
X = pd.read_csv("concat_features_final.csv")
with open("songs_scaler.pickle", "rb") as f: 
        scaler = pickle.load(f)
with open("kmeans_features.pickle", "rb") as f: 
        kmeans = pickle.load(f) 
with open("tsne_features.pickle", "rb") as f: 
        tsne = pickle.load(f) 

In [23]:
from config import *
import spotipy
import json
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd
import numpy as np
import random
import pprint
import pickle
from sklearn.preprocessing import StandardScaler
from sklearn.manifold import TSNE
from sklearn.cluster import KMeans
from IPython.display import display

# Assume the necessary variables and functions are already defined (e.g., tsne, kmeans, scaler, etc.)

def song_recommender():
    sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id=client_id,
                                                               client_secret=client_secret))
    song_title = input("What song title did you like listening to recently the most: ")
    song_artist = input("From which artist? ")
    query = "tracks: " + f"{song_title}" + " artist: " + f"{song_artist}"
    results = sp.search(q=query, limit=5)
    
    songs_dict = {"Artist": [], "Title": [], "id": []}
    for item in results['tracks']['items']:
        songs_dict['Title'].append(item['name'])
        songs_dict['Artist'].append(item['artists'][0]['name'])
        songs_dict['id'].append(item['id'])
    
    songs_df = pd.DataFrame(songs_dict)
    
    display(songs_df[["Artist", "Title"]])
    
    correct_song_index = int(input("Select the right song index: "))
    track_id = songs_df.iloc[correct_song_index, -1]

    list_of_ids = [track_id]
    user_song_audio_features = sp.audio_features(list_of_ids)
    user_song_audio_features_df = pd.DataFrame(user_song_audio_features)

    # Assuming 'user_song_audio_features' is a list of dictionaries
    user_song_audio_features_df = pd.DataFrame(user_song_audio_features)

# Filter out non-numeric columns
    numeric_columns = user_song_audio_features_df.select_dtypes(include=[np.number]).columns
    user_song_audio_features_df_numeric = user_song_audio_features_df[numeric_columns]

# Scale the numeric features
    user_song_audio_features_scaled_np = scaler.transform(user_song_audio_features_df_numeric)
    user_song_audio_features_scaled_df = pd.DataFrame(user_song_audio_features_scaled_np,
                                                  columns=user_song_audio_features_df_numeric.columns,
                                                  index=user_song_audio_features_df_numeric.index)


    user_song_tsne_features_np = tsne.embedding_
    user_song_tsne_features_df = pd.DataFrame(user_song_tsne_features_np, columns=["TSNE_1", "TSNE_2"])
    user_song_cluster = kmeans.predict(user_song_tsne_features_df)
    
    if track_id in concat_nhs_hs[concat_nhs_hs['Hot'] == "Y"]['id'].values:
        cpu_songs = concat_nhs_hs[(concat_nhs_hs['Hot'] == "Y")].sample(5)
        print("Nice!This is hot song. You will probably love listening to this popular songs as well:")
        display(cpu_songs[['Artist', 'Title']])
    else:
        cpu_songs = concat_nhs_hs[concat_nhs_hs['Hot'] == "N"].sample(5)
        print("Nice!This is not hot song You will probably love listening to these other songs as well:")
        display(cpu_songs[['Artist', 'Title']])
  
# # Example of usage
# answer = "Yes"
# while answer == "Yes":
#     song_recommender()
#     answer = input("Do you want another recommendation? (Yes/No)")
#     while answer not in ["Yes", "No"]:
#         print()
#         print("I said (Yes/No)!!!!!")
#         answer = input("Do you want another recommendation? (Yes/No)")

In [24]:
answer = "Yes"
while answer =="Yes":
    song_recommender()
    answer == input("Do you want another recomendation? (Yes/No)")
    while answer not in ["Yes","No"]:
        print()
        print("I said (Yes/No)!!!!!")
        answer == input("Do you want another recomendation? (Yes/No)")
      

What song title did you like listening to recently the most: In the End
From which artist? Linkin Park


,Artist,Title
0,Linkin Park,In the End
1,Black Veil Brides,In The End
2,Linkin Park,In the End
3,Tommee Profitt,In The End - Mellen Gi Remix
4,Linkin Park,In the End (Demo) - LPU Rarities


Select the right song index: 0
Nice!This is not hot song You will probably love listening to these other songs as well:


,Artist,Title
361,Tweet,Oops (Oh My) [feat. Missy Elliott]
429,Ashanti,Rock Wit U (Awww Baby)
2390,Meghan Trainor,Made You Look
1081,Calvin Harris,I'm Not Alone - Radio Edit
2143,Justin Bieber,Holy (feat. Chance The Rapper)


KeyboardInterrupt: Interrupted by user